<a href="https://colab.research.google.com/github/neelansh27/MNIST_model/blob/main/fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input,Conv2D, Dense, Flatten
from tensorflow.keras.models import Model

In [ ]:
fashion_mnist=tf.keras.datasets.fashion_mnist
(x_train,y_train),(x_test,y_test)=fashion_mnist.load_data()
x_train=x_train/255
x_test=x_test/255

In [ ]:
# total no. of outputs/categories
K=len(np.unique(y_train))
K

In [ ]:
x_train.shape

In [ ]:
# Since the Data is 2D, we have to convert it to 3D
# because CNN expects input of form "N x H x W x C"
x_train=np.expand_dims(x_train,-1) # Can also use .reshape method
x_test=x_test.reshape(*x_test.shape,1)
x_train.shape

In [ ]:
# Building the model
model=tf.keras.Sequential([
    Input(shape=x_train[0].shape),
    Conv2D(32,(3,3),strides=2,activation='relu'),
    Conv2D(64,(3,3),strides=2,activation='relu'),
    Conv2D(128,(3,3),strides=2,activation='relu'),
    Flatten(),
    Dense(512,activation='relu'),
    Dense(K,activation='softmax'),
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
stopper=tf.keras.callbacks.EarlyStopping()
res=model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=8,
    callbacks=[stopper],
    verbose=2)

In [ ]:
loss,acc=model.evaluate(x_test,y_test)
print(f'Accuracy:{acc*100:.2f}%\nLoss:{loss:.2f}')

In [ ]:
plt.plot(res.history['loss'],label='loss')
plt.plot(res.history['val_loss'],label='validation loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(res.history['accuracy'],label='Accuracy')
plt.plot(res.history['val_accuracy'],label='Validation accuracy')
plt.title('Accuracy')
plt.legend()
plt.show()

In [ ]:
yhat=model.predict(x_test).argmax(axis=1)
yhat

In [ ]:
from sklearn.metrics import confusion_matrix
pred_tab=confusion_matrix(y_test,yhat)

In [ ]:
import seaborn as sns
labels='''T-shirt/top
Trouser
Pullover
Dress
Coat
Sandal
Shirt
Sneaker
Bag
Ankle_boot'''.split()
sns.heatmap(data=pred_tab,annot=True,fmt='.0f',linecolor='black',
            clip_on=False,yticklabels=labels,xticklabels=labels,
            linewidth=0.5,cmap=plt.cm.Purples)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# See an image which was predicted wrong
wrong_pred=np.where(yhat!=y_test)[0]
id=np.random.choice(wrong_pred)
plt.imshow(x_test[id].reshape(28,28),cmap='gray')
plt.title(f'{labels[y_test[id]]} classified as {labels[yhat[id]]}')
plt.axis('off')
plt.show()